<a href="https://colab.research.google.com/github/maglesT/projects/blob/main/llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pdfminer.six
!pip install langchain_google_genai python-docx langchain


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
from pdfminer.high_level import extract_text
import re

def extract_text_with_pdfminer(pdf_path):
    return extract_text(pdf_path)

def clean_text(text):

    text = text.replace('\f', ' ')
    text = re.sub(r'\s+\n', '\n', text)
    text = re.sub(r'\n+', '\n', text)
    return text.strip()

def extract_sections(text):
    text = clean_text(text)


    pattern = r'^(Section\s+\d+:\s+.*)$'
    headings = [(m.start(), m.group(0).strip()) for m in re.finditer(pattern, text, re.MULTILINE)]

    sections = {}
    if headings:

        if headings[0][0] > 0:
            preamble = text[:headings[0][0]].strip()
            if preamble:
                sections["Title"] = preamble


        for i, (start, heading) in enumerate(headings):
            end = headings[i+1][0] if i+1 < len(headings) else len(text)
            section_text = text[start:end].strip()

            content = section_text[len(heading):].strip()
            sections[heading] = content
    else:
        sections["FullText"] = text
    return sections

# --- Main Pipeline ---


pdf_path = "pdf/ESS Switching System.pdf"
pdf_text = extract_text_with_pdfminer(pdf_path)
sections = extract_sections(pdf_text)





section_list_with_headings = [(heading, content) for heading, content in sections.items()]



In [5]:

formatted_sections = [f"{heading}\n\n{content}" for heading, content in section_list_with_headings]


print(formatted_sections[0])

FullText

Electronic switching system (ESS)
such
as digital
In telecommunications,  an electronic  switching  system (ESS)  is  a telephone
switch that  uses solid-state
electronics)
electronics,
and computerized common control, to interconnect telephone circuits for the
purpose of establishing telephone calls.
The  generations  of  telephone  switches  before  the  advent  of  electronic
switching  in  the  1950s  used  purely electro-mechanical relay  systems  and
analog  voice  paths.  These  early  machines  typically  utilized  the step-by-
step technique.  The  first  generation  of  electronic  switching  systems  in  the
1960s  were  not  entirely  digital  in  nature,  but  used reed  relay-operated
metallic paths or crossbar switches operated by stored program control (SPC)
systems.
Reed relays
Reed  relays  contain  a  reed  switch,  a  coil  for  creating  a  magnetic  field,  an
optional  diode  for  handling  back  EMF  from  the  coil,  and  an  encapsulating
package  wi

In [6]:
for section in formatted_sections:

    print(section)
    print()


FullText

Electronic switching system (ESS)
such
as digital
In telecommunications,  an electronic  switching  system (ESS)  is  a telephone
switch that  uses solid-state
electronics)
electronics,
and computerized common control, to interconnect telephone circuits for the
purpose of establishing telephone calls.
The  generations  of  telephone  switches  before  the  advent  of  electronic
switching  in  the  1950s  used  purely electro-mechanical relay  systems  and
analog  voice  paths.  These  early  machines  typically  utilized  the step-by-
step technique.  The  first  generation  of  electronic  switching  systems  in  the
1960s  were  not  entirely  digital  in  nature,  but  used reed  relay-operated
metallic paths or crossbar switches operated by stored program control (SPC)
systems.
Reed relays
Reed  relays  contain  a  reed  switch,  a  coil  for  creating  a  magnetic  field,  an
optional  diode  for  handling  back  EMF  from  the  coil,  and  an  encapsulating
package  wi

In [ ]:
import os
import re
import requests
from io import BytesIO
from PIL import Image
from IPython.display import display
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH


from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document as LC_Document
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI

# Setup API Keys
os.environ["GOOGLE_API_KEY"] = "Insert  API Key Here"  # Replace with your Gemini API key
google_api_key = "Insert Search API Key Here"          # Google Custom Search API key
cx = "648b68ccf3907408b"                    # Your Custom Search Engine ID

# Initialize the LLM (Gemini 2.0 Flash)
llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)




def google_image_search(query, api_key, cx, num=1):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": api_key,
        "cx": cx,
        "searchType": "image",
        "num": num
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    results = response.json()
    if "items" in results and len(results["items"]) > 0:
        return results["items"][0]["link"]
    else:
        return None


notes_per_section = {}      # section index -> detailed notes text
images_per_section = {}     # section index -> image URL

# Process each section
for idx, section in enumerate(formatted_sections, start=1):
    print(f"\nProcessing Section {idx}...")

    #  Stage 1: Generate Detailed Lecture Notes for the section
    text_prompt = f"""
Please transform the following text section into detailed, elaborated lecture notes for students.
Expand on the content by adding clear explanations and extra details to enhance understanding.
Format your output as follows:
- Start each main heading with "# " (title case).
- Start each subheading with "## " on a new line.
- Use "* " to begin each bullet point on its own line.
- Write normal paragraphs as plain text.
- Use proper Unicode for any subscripts/superscripts (e.g., output CO₂ as "CO₂").
- Do not include any extra text or instructions beyond the elaborated lecture notes.

Text:
{section}
    """
    detailed_notes = llm(text_prompt).strip()
    print("Detailed notes generated.")
    notes_per_section[idx] = detailed_notes

    # Stage 2: Generate Image Search Query for the section
    image_prompt = f"""
Based on the following section's heading and key points, generate a concise image search query that would return a relevant image for this topic.

Section:
{section}

Image Search Query:
    """
    image_query = llm(image_prompt).strip()
    print("Generated image query:", image_query)

    # Use the generated query to search for an image.
    image_url = google_image_search(image_query, google_api_key, cx)
    print("Retrieved image URL:", image_url)
    images_per_section[idx] = image_url


    if image_url:
        try:
            img_response = requests.get(image_url)
            img_response.raise_for_status()
            img_data = BytesIO(img_response.content)
            img = Image.open(img_data)
            display(img)
        except Exception as e:
            print("Error displaying image:", e)
    else:
        print("PTO")

#  Concatenate all detailed notes for summarization
all_notes = ""
for idx in sorted(notes_per_section.keys()):
    all_notes += "\n" + notes_per_section[idx]

#  Generate an overall summary using the detailed notes
doc_for_summary = LC_Document(page_content=all_notes)
custom_summarize_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "You are an expert in this subject. Read the text below and produce a comprehensive, detailed summary. "
        "Include the main points, examples, and any significant takeaways. "
        "Make sure the summary is clear, concise, and covers all important details.\n\n"
        "TEXT:\n{text}\n\n"
        "DETAILED SUMMARY:"
    ),
)
summarize_chain = load_summarize_chain(
    llm,
    chain_type="stuff",
    prompt=custom_summarize_prompt,
)
summary = summarize_chain.run([doc_for_summary])
print("\n=== Overall Summary ===\n", summary)

# Create the Summary Word Document (text only)
summary_doc = Document()

# Style the summary document
summary_heading_style = summary_doc.styles["Heading 1"]
summary_heading_style.font.name = "Arial"
summary_heading_style.font.size = Pt(16)
summary_heading_style.font.bold = True
summary_normal_style = summary_doc.styles["Normal"]
summary_normal_style.font.name = "Calibri"
summary_normal_style.font.size = Pt(11)

summary_doc.add_paragraph("Summary", style="Heading 1").alignment = WD_ALIGN_PARAGRAPH.CENTER

# Split the summary into sentences and add each as a paragraph for readability.
sentences = re.split(r'(?<=[.!?])\s+', summary.strip())
for sentence in sentences:
    if sentence.strip():
        p = summary_doc.add_paragraph(sentence.strip(), style="Normal")
        p.alignment = WD_ALIGN_PARAGRAPH.LEFT

summary_doc_filename = "fromsoft_summary.docx"
summary_doc.save(summary_doc_filename)
print(f"Summary document generated: {summary_doc_filename}")

# Create the Main Detailed Notes Word Document with Images
main_doc = Document()

# Customize styles for the main document.
main_heading1 = main_doc.styles["Heading 1"]
main_heading1.font.name = "Arial"
main_heading1.font.size = Pt(16)
main_heading1.font.bold = True

main_heading2 = main_doc.styles["Heading 2"]
main_heading2.font.name = "Arial"
main_heading2.font.size = Pt(14)
main_heading2.font.bold = True

main_normal = main_doc.styles["Normal"]
main_normal.font.name = "Calibri"
main_normal.font.size = Pt(11)

# For each section, add the detailed notes and then the image.
for idx in sorted(notes_per_section.keys()):
    # Add the detailed notes.
    section_text = notes_per_section[idx]

    # Split the section text into lines; assume the first line is a heading.
    lines = section_text.strip().split("\n")
    if lines:
        heading_text = lines[0].lstrip("# ").strip()
        main_doc.add_heading(heading_text, level=1)
        for line in lines[1:]:
            if line.strip():
                # Remove bullet markers if present.
                text = line.lstrip("* ").strip()
                main_doc.add_paragraph(text, style="Normal")

    # Add the corresponding image if available.
    image_url = images_per_section.get(idx)
    if image_url:
        try:
            img_response = requests.get(image_url)
            img_response.raise_for_status()
            img_data = BytesIO(img_response.content)
            main_doc.add_picture(img_data, width=Inches(4))
        except Exception as e:
            print(f"Error adding image for section {idx}:", e)
    else:
        main_doc.add_paragraph("No image available for this section.", style="Normal")

    # Add a page break after each section.
    main_doc.add_page_break()

main_doc_filename = "fromsoft_notes.docx"
main_doc.save(main_doc_filename)
print(f"Main detailed notes document generated: {main_doc_filename}")



Processing Section 1...


E0000 00:00:1759578388.432544   65631 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised PermissionDenied: 403 Generative Language API has not been used in project 265244582491 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/generativelanguage.googleapis.com/overview?project=265244582491 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "serviceTitle"
  value: "Generative Language API"
}
metadata {
  key: "containerInfo"
  value: "265244582491"
}
metadata {
  key: "consumer"
  value: "projects/265244582491"
}
metadata {
  key: "activationUrl"
  value: "https://console.d

In [11]:
## Redundant Code
## from google.colab import files
## files.download('final_notes_with_images_new.docx')